In [ ]:
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: FSharp.Data"
#r "nuget: Deedle"


In [ ]:
open System
open XPlot.Plotly
open FSharp.Data
open Microsoft.DotNet.Interactive.Formatting
open Deedle


In [ ]:
//get an array of 3 occurences
let dat = [| 10; 20; 30 |]

//print the array 
Array.iter (fun x -> printfn "%d" x) dat

//let now simulate a larger dataset of 100 random numbers
//one commom functionality to generate normal ¨normally distributed numbers is the Box-Muller transform
// Setting a seed for the random number generator
let seed = 154
let random = Random(seed)

let boxMullerTransform (rand : Random) =
    let u1 = random.NextDouble()
    let u2 = random.NextDouble()
    let z0 = Math.Sqrt(-2.0 * Math.Log(u1)) * Math.Cos(2.0 * Math.PI * u2)
    z0

In [ ]:
let values = Array.init 10 (fun _ -> boxMullerTransform(random))
// Printing the array
Array.iter (fun value -> printfn "%f" value) values

let histogram = 
    Histogram(
        x = values,
        autobinx = true
    )

let layout = Layout(title = "Normal Distribution")

Chart.Plot([histogram])
|> Chart.WithTitle("Normal Distribution")




In [ ]:
let z = 3
let x = [1; 2; 3; 4; 5]
let y = [1; 4; 9; 16; 25]
printfn $"z = {z}"

let scatter = Scattergl(x = x, y = y, mode = "markers")

Chart.Plot([scatter])
|> Chart.WithTitle("Scatter Plot")
|> Chart.WithXTitle("X-Axis")
|> Chart.WithYTitle("Y-Axis")

In [ ]:
let today = DateTime.Today
let startDate = new DateTime(2000, 1, 1) // Example start date
let url = 
        sprintf "https://query1.finance.yahoo.com/v7/finance/download/^GSPC?period1=0&period2=9999999999&&interval=1d&events=history&includeAdjustedClose=true" 
                


In [ ]:
let fetchSP500Data () =      
    try
        Some(Http.RequestString(url))
    with
    | :? System.Net.WebException as e -> 
        printfn "An error occurred: %s" e.Message
        None

match fetchSP500Data () with
| Some data -> 
    printfn "%s" data // Prints the CSV data
| None -> 
    printfn "Failed to fetch S&P 500 data"

In [ ]:
let saveDataToCsv (filePath: string) (data: string) =
    System.IO.File.WriteAllText(filePath, data)

let sp500Data = fetchSP500Data ()
match sp500Data with
| Some data -> 
    let filePath = __SOURCE_DIRECTORY__ + "/../data/sp500data.csv" 
    saveDataToCsv filePath data
    printfn "Data saved to %s" filePath
| None -> 
    printfn "Failed to fetch or save S&P 500 data"

In [ ]:
let sp500 =
    CsvFile
        .Load(__SOURCE_DIRECTORY__ + "/../data/sp500data.csv")
        .Cache()

In [ ]:
// Print the prices in the HLOC format
for row in sp500.Rows |> Seq.truncate 10 do
    printfn "HLOC: (%s, %s, %s)" (row.GetColumn "High") (row.GetColumn "Low") (row.GetColumn "Date")


In [ ]:
let filePath = __SOURCE_DIRECTORY__ + "/../data/sp500data.csv" 
let df = Frame.ReadCsv(filePath)

In [ ]:
let df_Ord = 
  df
  |> Frame.indexRowsDate "Date"
  |> Frame.sortRowsByKey